## TO DOs:
* aggiungere il controllo sul website http ovunque
* attendere il feedback sulle news inviato per mail (14 agosto)
* attendere la lista dei tags rivisti e rivedere poi il processo di assegnazione (13 agosto)
* caricare tutto su postgre 
* Nel processo di update del database aggiungere l'opzione di dubbio e quindi segnalare startup da controllare manualmente

### LIBRARIES AND GLOBAL VARIABLES

In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import requests
from bs4 import BeautifulSoup
import openai
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import json
import logging
import re
from googlesearch import search
import hashlib
from dotenv import load_dotenv
import os
from urllib.parse import urlparse
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import MoveTargetOutOfBoundsException, ElementNotInteractableException, StaleElementReferenceException
import random



# Initialize WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
chrome_options.add_argument('--headless')  # Run headless Chrome
service = Service(r'C:\Users\kevin\Desktop\RetailHub\new_software\chromedriver\chromedriver-win64\chromedriver.exe')  # Update the path to your chromedriver
driver = webdriver.Chrome(service=service, options=chrome_options)


# Your Airtable API key and base ID
api_key_airtable = 'patRHlt60PBL6jlV5.f2b9afed49bf23d9ab73cee6b283af76d3b4a9ebc0619002c6f104cc84382f2c'
base_id = 'appUMz37tbS84AKX4'
table_name = 'integrated_table'

# Airtable API endpoint for the specified table
url_airtable = f'https://api.airtable.com/v0/{base_id}/{table_name}'

# Headers for the API request
headers = {
    'Authorization': f'Bearer {api_key_airtable}',
    'Content-Type': 'application/json'
}

# Carica le variabili d'ambiente dal file .env
load_dotenv()

# Valorizza la chiave API con la variabile dell'ambiente
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configura il logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Android 11; Mobile; rv:89.0) Gecko/89.0 Firefox/89.0",
    "Mozilla/5.0 (Linux; Android 11; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Mobile Safari/537.36",
]


### FUNCTIONS

In [7]:
def reset_driver():
    # Reinizializza il WebDriver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Ensure GUI is off
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    service = Service(r'C:\Users\kevin\Desktop\RetailHub\new_software\chromedriver\chromedriver-win64\chromedriver.exe')  # Aggiorna il percorso al tuo chromedriver
    
    # Restituisci il nuovo driver
    return webdriver.Chrome(service=service, options=chrome_options)


def initialize_driver(headless=True, window_size=(1920, 1080)):
    """Inizializza e restituisce un driver configurato con opzioni avanzate."""
    
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument("--headless")  # Modalità headless opzionale
    
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--disable-extensions")
    
    # Seleziona un User-Agent casuale
    user_agent = random.choice(USER_AGENTS)
    chrome_options.add_argument(f"user-agent={user_agent}")
    
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    
    # Inizializza il WebDriver
    driver = webdriver.Chrome(options=chrome_options)
    
    # Imposta la dimensione della finestra
    driver.set_window_size(*window_size)
    
    # Esegui script per mascherare l'uso di webdriver
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    return driver
    


def get_all_text(url, max_depth=2, current_depth=0, visited=None, only_main=False, max_chars=50000):
    
    if not url.startswith(('http://', 'https://')):
        # Se manca lo schema, aggiunge "https://"
        url = 'https://' + url

    text = ''
    if visited is None:
        visited = set()

    if current_depth > max_depth or url in visited:
        return ""

    visited.add(url)
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove script and style elements
        for script in soup(['script', 'style']):
            script.decompose()

        # Get text from the current page
        main_text = soup.get_text(separator=' ', strip=True)

        # ritorna solo il main
        if only_main is True:
            return main_text
        else:
            text = main_text

            # Check if the text has reached the limit
            if len(text) >= max_chars:
                return text[:max_chars]

            # Find all links on the current page
            links = [a['href'] for a in soup.find_all('a', href=True)]

            # Base URL to resolve relative links
            base_url = requests.utils.urlparse(url)._replace(path="", query="", fragment="").geturl()

            # Recursively get text from subpages
            for link in links:
                if not link.startswith('http'):
                    link = requests.compat.urljoin(base_url, link)
                
                sub_text = get_all_text(link, max_depth, current_depth + 1, visited)
                
                # Append the subpage text if within the limit
                if len(text) + len(sub_text) > max_chars:
                    text += sub_text[:max_chars - len(text)]
                    break
                else:
                    text += " " + sub_text

            return text[:max_chars]

    except requests.RequestException:
        return text



def analyze_text_with_gpt(text, initial_data):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, api_key=openai.api_key)

    system_template = ("You are a helpful assistant that extracts data from the user input.")

    template = '''You will be given all the text from all the website pages of one startup. 
                    Your task is to identify and return several pieces of information from the input text. 
                    Please format your answer (in English, mandatory) as follows (it MUST be a numeric pointed list):

                    1. Name: The name of the brand/startup.
                    2. Business_model: A couple of words regarding the business model of the startup.
                    3. Business_description: At least 250 detailed words, in third person, with a title summarizing the business of the startup.
                    4. Founding_year: The year the startup was founded.
                    5. Founders: The names and roles of the founders.
                    6. Product_description: A detailed description of the startup's main products or services, in third person, at least 250 words as bullet points.
                    7. City: The city where the startup is headquartered.
                    8. Country: The country where the startup is headquartered.
                    9. Facebook_url: The startup's official Facebook page URL.
                    10. Notable_achievements_awards: List any significant achievements or awards the startup has received.
                    11. Target_markets: The target markets the startup aims to serve (europe, america, ecc....).
                    12. Company_type: The type of company (e.g., LLC ecc...).
                    13. Clients: List of notable clients.
                    14. Tags: Relevant tags or keywords associated with the startup. Must be a subset of the followings: Advertising, Agricultural Tech, Agriculture, Analytics, APIs, AR/VR Technologies, Arts, Asset Management, Automation, Automotive Tech, B2B, Banking, Big Data, Blockchain, Business Development, Business Intelligence, Business Productivity, Cloud Computing, Communication Technology, Computer Vision, Consumer Electronics, Content Production, CRM, Customer Service, Customer Support, Cybersecurity, Data Privacy, Developer Tools, Digital Marketing, E-Commerce, E-Government, EdTech, Education Technology, Energy Tech, Entertainment, Environmental Tech, Event Management, Fashion, Fashion Tech, Finance, Financial Services, FinTech, Food and Beverage Industry, Gamification, Gaming, Geospatial Services, Green Tech, Health and Wellness, HealthTech, Home Improvement, Hospitality, Human Resources, Information Technology, Infrastructure Management, Innovation, Insurance, International, Investment, Learning, Legal Compliance, Localization, Logistics, Manufacturing, Marketing, Marketing Automation, Marketplaces, Materials Science, Media, Mobile Technology, Mobility Tech, Networking, Non-profit Sector, On-Demand Services, Outdoor, Payments, Personal Care, Personalization, Quality Assurance, Real Estate, Recruitment, Recycling, Research, Retail Technology, Revenue Models, Robotics, SaaS Solutions, Sales Growth, Security, SEO, Smart Home Tech, Social Impact, Social Networking, Software Development, Software Services, Sports Tech, Streaming Services, Internet of Things (IoT), Subscription Services, Supply Chain, Sustainability, Telecommunications, Travel, Veterinary Services, Waste Management, Wireless Tech, Workforce Management, CIRCULAR ECONOMY, Packaging, Shelf intelligence, Virtual Try-On, BNPL, Return Management, Visual Merchandising, Demand planning, Pricing Solution, size guide, Autonomous Store, Fraud prevention, Digital Checkout, workflow management, Rental Platform, Autonomous Store, SHELF MANAGEMENT, Eyewear Experience, shopping experience, ecommerce, 3D Technology, Accessibility Tech, Advanced Manufacturing, AI, NRF24, Digital Wallet, Retail Marketing, Customer Experience, Smart Shelf, In-Store Technology, Smart Tags, Task Management, SaaS Platform, Neuroscience Solutions, Sensory Marketing, Real-time Monitoring, AI Checkout, Fast Checkout, Seamless Checkout, Shelf Monitoring, Store Optimization, Physical Storage, Real Estate, Online Payments, Blockchain Integration, Neuroscience, Footwear Technology, digital transformation, Product Design, Real-Time Analytics, Robotics Solutions, Shelf Stocking, Data Automation, Real-time Solutions, CRM, Energy, Computer Vision, Trend Forecasting, Seamless Integration, Immersive Experiences, loyalty program, In-Video Checkout, Personalized Recipes, In-Store Insights, store locator, loyalty automation, product passport, In-store management, Click&Collect, Demand Forecasting, Checkout Alternative, crypto, On-Demand, Fleet Management, Facial Recognition, Gift Cards, Metaverse, In-store, Self-ordering, Buy Now Pay Later, smart planning, 3d commerce, dynamic inventory, pop-up, zero emission, waste, Pick Up, Picking, carbon neutrality, web 3.0, Customer acquisition, Free delivery, customer loyalty, data analytics, retail analytics, virtual fitting room, virtual try-on, store analytics, elearning, self-checkout, Pay later, Checkout Automation'.
                    15. Phone_number: The contact phone number of the startup.
                    16. Technologies_used: The areas of interest and technologies the startup utilizes.
                    17. Address: The complete address of the startup.
                    18. Region: The region where the startup is located (depends on the country).
                    19. Number_of_employees: The total number of employees.
                    20. Main_investors: The main investors in the startup.
                    21. Number_of_investors: The total number of investors (based on the previous answer).
                    22. Investment_funds: The investment funds involved with the startup.
                    23. Exit_summary: A summary of the exit strategy or past exits.
                    24. Total_funding: The total funding received by the startup (in currency).
                    25. Advisors: List of advisors to the startup.
                    26. LinkedIn_URL: The startup's official LinkedIn page URL.
                    27. IPO_summary: Details about any IPOs.
                    28. Value_of_the_startup: The valuation of the startup.
                    29. Number_of_patents: The number of patents granted to the startup.
                    30. Number_of_trademarks: The number of trademarks registered by the startup.
                    31. Operating_status: startup, scaleup ecc.....
                    32. Type_of_latest_investment: The type of the latest investment received.
                    33. Acquired_by: The entity that acquired the startup, if applicable.
                    34. Video_demo: URL to any video or demo available on the startup website.
                    35. Website: The startup's official website URL.
                    36. Revenue: the startup's annual revenue.
                    37. Growth_rate: the growth rate of the last year for the startup.
                    38. Logo_url: the url of the logo of the brand.

                    If any of this information is not available in the text, try to figure it out on your own, otherwise write only the word NULL for that item and nothing else (mandatory).
                    Additionally, you will be provided with a dictionary containing initial information about the startup. Please use this information as is and do not modify it. The dictionary is as follows: {initial_data}.
                    The text to work on is the following: {data}.'''


    # Create the chat prompt
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    user_prompt = PromptTemplate(template=template, input_variables=["data"])
    user_message_prompt = HumanMessagePromptTemplate(prompt=user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, user_message_prompt])
    llm_chain = LLMChain(prompt=chat_prompt, llm=llm, verbose=False)

    result = llm_chain.run({"data": text, "initial_data": initial_data})
    return result



def get_product_description(text):
    start_keyword = "6. Product description:"
    end_keyword = "7."

    start_index = text.find(start_keyword)
    end_index = text.find(end_keyword)

    if start_index != -1 and end_index != -1:
        # Extract and clean the product description
        product_description = text[start_index + len(start_keyword):end_index].strip()
        return product_description
    else:
        return "NULL"



def convert_to_dict(text):
    # Split the text into lines
    lines = text.split('\n')
    
    # Dictionary to hold the results
    data_dict = {}
    
    # Regex to match the keys
    key_pattern = re.compile(r'^\d+\. ([^:]+):')
    
    # Current key
    current_key = None
    
    # Process each line
    for line in lines:
        key_match = key_pattern.match(line)
        if key_match:
            # Found a new key
            current_key = key_match.group(1).strip()
            value = line[key_match.end():].strip()
            data_dict[current_key] = value
        elif current_key:
            # Append the line to the current key's value
            data_dict[current_key] += ' ' + line.strip()
    
    return data_dict



def is_json(response):
    try:
        json.loads(response.text)
    except ValueError:
        return False
    return True




# Function to upload a single record to Airtable
def upload_record_to_airtable(record, url, headers):
    data = {
    "records": [
        {
            "fields": {
            "Name": record.get("Name", "NULL"),
            "Business_model": record.get("Business_model", "NULL"),
            "Business_description": record.get("Business_description", "NULL"),
            "Founding_year": record.get("Founding_year", "NULL"),
            "Founders": record.get("Founders", "NULL"),
            "Product_description": record.get("Product_description", "NULL"),
            "City": record.get("City", "NULL"),
            "Country": record.get("Country", "NULL"),
            "Facebook_url": record.get("Facebook_url", "NULL"),
            "Notable_achievements_awards": record.get("Notable_achievements_awards", "NULL"),
            "Target_markets": record.get("Target_markets", "NULL"),
            "Company_type": record.get("Company_type", "NULL"),
            "Clients": record.get("Clients", "NULL"),
            "Tags": record.get("Tags", "NULL"),
            "Phone_number": record.get("Phone_number", "NULL"),
            "Technologies_used": record.get("Technologies_used", "NULL"),
            "Address": record.get("Address", "NULL"),
            "Region": record.get("Region", "NULL"),
            "Number_of_employees": record.get("Number_of_employees", "NULL"),
            "Main_investors": record.get("Main_investors", "NULL"),
            "Number_of_investors": record.get("Number_of_investors", "NULL"),
            "Investment_funds": record.get("Investment_funds", "NULL"),
            "Exit_summary": record.get("Exit_summary", "NULL"),
            "Total_funding": record.get("Total_funding", "NULL"),
            "Advisors": record.get("Advisors", "NULL"),
            "LinkedIn_URL": record.get("LinkedIn_URL", "NULL"),
            "IPO_summary": record.get("IPO_summary", "NULL"),
            "Value_of_the_startup": record.get("Value_of_the_startup", "NULL"),
            "Number_of_patents": record.get("Number_of_patents", "NULL"),
            "Number_of_trademarks": record.get("Number_of_trademarks", "NULL"),
            "Operating_status": record.get("Operating_status", "NULL"),
            "Type_of_latest_investment": record.get("Type_of_latest_investment", "NULL"),
            "Acquired_by": record.get("Acquired_by", "NULL"),
            "Video_demo": record.get("Video_demo", "NULL"),
            "Website": record.get("Website", "NULL"),
            "Revenue": record.get("Revenue", "NULL"),
            "Growth_rate": record.get("Growth_rate", "NULL"),
            "Logo_url": record.get("Logo_url", "NULL"),
            "Key" : record.get("Key", "NULL"),
            "google_news_urls": record.get("google_news_urls", "NULL")#,
            #"timestamp": record.get("timestamp", "NULL")
                    }
        }
    ]

}
    response = requests.post(url, headers=headers, data=json.dumps(data))
    logger.info(f"Response status code: {response.status_code}")
    if response.status_code == 200:
        if is_json(response):
            try:
                response_json = response.json()
                #logger.info(f"Record added: {data['Name']}")
                #return response_json
            except json.JSONDecodeError as e:
                logger.error(f"Failed to parse response JSON: {e}")
                logger.error(f"Response text: {response.text}")
        else:
            logger.error("Response is not JSON.")
            logger.error(f"Response text: {response.text}")
    else:
        logger.error(f"Failed to add record: {response.status_code}")
        logger.error(f"Response text: {response.text}")

    return None





##################################
######## NEWS RETRIEVAL #########
##################################


def google_news_search(query, num_results=10):
    try:
        return list(search(query, num=num_results, stop=num_results, tbs='qdr:y'))  # 'tbs=qdr:y' filters for past year
    except Exception as e:
        print(f"Error during Google News search: {e}")
        return []

def summarize_news_articles(articles_text):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, api_key=openai.api_key)

    # Prompt template per il riassunto
    template = '''
    Below is the text from multiple articles about a startup. Provide a concise summary of only 3 of the articles, separated by ";;;":
    Choose carefully which ones to summarize without repeating yourself, and prioritize articles that discuss different aspects.
    Include the name of the source, the article's publication date, and the author's name (if available; if not, leave it out).
    The articles must be written in English and only include summaries of articles that mention and discuss the startup.
    If there are fewer than 3 relevant articles, summarize fewer.
    {articles_text}
    '''


    # Creazione del prompt
    user_prompt = PromptTemplate(template=template, input_variables=["articles_text"])
    chat_prompt = ChatPromptTemplate.from_messages([HumanMessagePromptTemplate(prompt=user_prompt)])
    llm_chain = LLMChain(prompt=chat_prompt, llm=llm, verbose=False)

    # Chiamata all'API di GPT-4o-mini per ottenere il riassunto
    result = llm_chain.run({"articles_text": articles_text})
    
    return result

def news(startup_urls):
    for startup_url in startup_urls:
        print(f"Processing {startup_url}...")
        
        # Step 1: Search Google News for related articles 
        query = f"{startup_url} startup news"
        search_results = google_news_search(query)
        
        if not search_results:
            print(f"No search results found for {startup_url}")
            continue

        # Step 2: Scrape text from each found article
        concatenated_text = ""
        for result in search_results:
            concatenated_text += get_all_text(result, only_main=True) + " ---- END ---- "

        # Step 3: Generate summaries for the concatenated articles
        summarized_text = summarize_news_articles(concatenated_text)
    
    return summarized_text




    


##################################
# CREAZIONE DELLA CHIAVE UNIVOCA #
##################################

# Funzione per preprocessare la stringa
def preprocess_string(s):
    s = s.lower()  # Converti in minuscolo
    s = re.sub(r'[^\w.]', '', s)  # Remove all non-alphanumeric characters except dot (.)
    return s

# Funzione per normalizzare un URL
def normalize_url(url):
    parsed_url = urlparse(url)
    # Prendi solo il netloc (dominio) e il percorso (escludi http/https e www)
    normalized_url = parsed_url.netloc.replace('www.', '') + parsed_url.path
    # Rimuovi eventuali slash finali
    normalized_url = normalized_url.rstrip('/')
    return normalized_url

# Funzione per generare una chiave univoca utilizzando solo il website
def generate_unique_key(website):
    website = preprocess_string(normalize_url(website))
    unique_key = hashlib.sha256(website.encode()).hexdigest()
    return unique_key


#############################
# ESTRAZIONE LINKS INTERESSANTI
#############################
def analyze_links_with_gpt(links):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, api_key=openai.api_key)

    # Template per il prompt di sistema
    system_template = (
        "Sei un assistente esperto che aiuta a identificare link web che contengono liste o ranking di startup. "
        "Rispondi con una lista di link che parlano di liste o ranking di startup, uno per riga, senza numerazione o testo aggiuntivo."
    )

    # Template per il prompt dell'utente
    template = '''
    Ecco una lista di link web. Per favore, identifica quali di questi contengono liste o ranking di startup e restituisci solo i link:
    {links}
    '''

    # Creazione del prompt
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    user_prompt = PromptTemplate(template=template, input_variables=["links"])
    user_message_prompt = HumanMessagePromptTemplate(prompt=user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, user_message_prompt])
    llm_chain = LLMChain(prompt=chat_prompt, llm=llm, verbose=False)

    # Prepara i link come una stringa unica separata da nuovi linee
    links_str = "\n".join(links)

    # Chiamata all'API di GPT-4 per ottenere il risultato filtrato
    result = llm_chain.run({"links": links_str})
    
    # Post-processing: Rimuovi eventuali spazi vuoti e filtra solo le righe che contengono link validi
    filtered_links = [link.strip() for link in result.split("\n") if link.strip().startswith("http")]
    
    return filtered_links



###############################
# CREAZIONE DIZIONARIO STARTUP
###############################

dict_example = [
                {'name': 'startup 1',
                 'website': 'website 1',
                 'venture_radar_profile': 'profile 1'},
                {'name': 'startup 2',
                 'website': 'website 2',
                 'venture_radar_profile': 'profile 2'}
                 ]

# Funzione per analizzare il contenuto della pagina della startup
def analyze_startup_page_with_gpt(html_content, dict_example):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, api_key=openai.api_key)

    # Template per il prompt di sistema
    system_template = (
        "Sei un assistente esperto che analizza il contenuto di una pagina web per estrarre le informazioni principali "
        "relative a una startup, come il nome della startup, l'URL del sito della startup e l'URL del profilo della startup su Venture Radar. "
        "Restituisci queste informazioni come lista python di dizionari, senza la formattazione markdown: rispondi solamente con quello richiesto."
        "il seguente è un esempio di output: {dict_example}"
        )

    # Template per il prompt dell'utente
    template = '''
    Qui di seguito c'è il contenuto HTML di una pagina web che descrive una o più startup. 
    Estrai e restituisci le seguenti informazioni per ogni startup trovata, nel formato richiesto:
    - Nome della startup
    - URL del sito della startup
    - URL del profilo della startup su Venture Radar
    
    Contenuto HTML della pagina:
    {html_content}
    '''

    # Creazione del prompt
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    user_prompt = PromptTemplate(template=template, input_variables=["html_content", "dict_example"])
    user_message_prompt = HumanMessagePromptTemplate(prompt=user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, user_message_prompt])
    llm_chain = LLMChain(prompt=chat_prompt, llm=llm, verbose=False)

    # Chiamata all'API di GPT-4 per ottenere il risultato analizzato
    result = llm_chain.run({"html_content": html_content, "dict_example": dict_example})
    
    try:
        # Tenta di caricare il risultato come JSON
        startup_data = eval(result)
        return startup_data
    except json.JSONDecodeError:
        print("Failed to eval generated dict, returning raw result")
        print("Raw result:", result)
        return result  # Ritorna il risultato grezzo per ulteriori analisi


##############################
## LINK PROCESSING VENTURE
##############################

# Funzione per ottenere il link al sito web dalla pagina Venture Radar
def get_website_from_ventureradar(link):
    # Usa la funzione per inizializzare o resettare il driver
    driver = reset_driver()
    driver.get(link)
    time.sleep(2)  # Attendi il caricamento della pagina
    pageContent = driver.page_source
    soup = BeautifulSoup(pageContent, 'html.parser')
    website_div = soup.find('div', id='i_d_CompanyWebsiteLink')
    if website_div and website_div.find('a', href=True):
        return website_div.find('a')['href']
    return None




def is_valid_url(url, expected_domain=None):
    try:
        result = re.match(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', url)
        if expected_domain:
            return result and expected_domain in url
        return result is not None
    except:
        return False


def process_startups(startup_data):
    processed_startups = {}

    # Itera su ogni startup nel dizionario
    for startup in startup_data.get('page_1', []):
        startup_name = startup.get('name')
        startup_url = startup.get('website')
        venture_radar_url = startup.get('venture_radar_profile')

        # Caso 1: Sia il sito web che il sito interno sono valorizzati
        if startup_url and venture_radar_url:
            if not is_valid_url(venture_radar_url, "ventureradar.com") or startup_url == venture_radar_url:
                # Se il sito interno non è valido o è uguale al sito web, ricostruiamo il sito interno
                venture_radar_url = f"https://www.ventureradar.com/organisation/{startup_name.replace(' ', '%20')}"
            processed_startups[startup_name] = {
                "Startup URL": startup_url,
                "Venture Radar URL": venture_radar_url
            }

        # Caso 2: Solo il sito interno è valorizzato
        elif venture_radar_url:
            if is_valid_url(venture_radar_url, "ventureradar.com"):
                # Recuperiamo il sito web dalla pagina Venture Radar
                startup_url = get_website_from_ventureradar(venture_radar_url)
                if startup_url:
                    processed_startups[startup_name] = {
                        "Startup URL": startup_url,
                        "Venture Radar URL": venture_radar_url
                    }

        # Caso 3: Solo il sito web è valorizzato
        elif startup_url:
            # Costruiamo il sito interno
            venture_radar_url = f"https://www.ventureradar.com/organisation/{startup_name.replace(' ', '%20')}"
            processed_startups[startup_name] = {
                "Startup URL": startup_url,
                "Venture Radar URL": venture_radar_url
            }

        # Caso 4: Nessuno dei due URL è valorizzato
        else:
            pass
            # Non facciamo nulla, la startup non viene inclusa

    return processed_startups



##############################
### CAPTCHA RESOLVING
##############################

def human_like_scroll(driver, pause_time=2):
    """Simula lo scorrimento umano con pause casuali."""
    total_height = driver.execute_script("return document.body.scrollHeight")
    
    for i in range(1, total_height, random.randint(300, 600)):
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(random.uniform(pause_time - 1, pause_time + 1))

def safe_mouse_movement(driver):
    """Muove il mouse su un elemento sicuro per evitare errori di movimento."""
    actions = ActionChains(driver)
    try:
        element = driver.find_element(By.TAG_NAME, 'body')
        actions.move_to_element(element).perform()
        time.sleep(random.uniform(0.5, 1.5))
    except MoveTargetOutOfBoundsException:
        print("Move target out of bounds, skipping mouse movement.")

def random_click(driver):
    """Simula un click casuale su un elemento interattivo."""
    elements = driver.find_elements(By.XPATH, "//a | //button")
    interactive_elements = []
    
    for element in elements:
        try:
            if element.is_displayed() and element.is_enabled():
                interactive_elements.append(element)
        except (ElementNotInteractableException, StaleElementReferenceException):
            continue

    if interactive_elements:
        random.choice(interactive_elements).click()

### GROWJO

In [ ]:
# Initialize the WebDriver
base_url2 = "https://growjo.com/"


# Loop through pages from 2 to 195
for idxx, page_num in enumerate(range(1, 2)):
    page_url = f"https://growjo.com/home/{page_num}"
    print('Scraped page:', idxx+1)

    try:
        # Usa la funzione per inizializzare o resettare il driver
        driver = reset_driver()
        # Fetch the page
        driver.get(page_url)
        pageContent = driver.page_source
        soup = BeautifulSoup(pageContent, 'html.parser')

        # Find the main exhibitors div
        startup_containers = soup.find_all('tr', class_='jss33 jss35', role='checkbox', tabindex='-1')

        # For each row in the table
        for idxxx, row in enumerate(startup_containers):
            print('scraping of startup number', 50 * (idxx) + (idxxx + 1) )

            data = []
            try:
                # Extract ranking
                ranking = row.find('div', class_='ranking-wrapper').text.strip() if row.find('div', class_='ranking-wrapper') else ''
                
                # Extract company name and link
                company_cell = row.find_all('td', class_='text-start')[0]
                company_name = company_cell.find('a', href=lambda x: x and x.startswith('/company/')).text.strip() if company_cell.find('a') else ''
                print(company_name)
                company_link = company_cell.find('a', href=lambda x: x and x.startswith('/company/'))['href'] 
                
                # Extract location details
                location_cell = row.find_all('td', class_='text-start')[1]
                location = location_cell.find('a', class_='custom-anchor')['href'].split('/')[-1] if location_cell.find('a') else ''
                
                # Extract country
                country = row.find_all('td', class_='text-start')[2].text.strip() if len(row.find_all('td', class_='text-start')) > 2 else ''
                
                # Extract funding details
                funding = row.find_all('td', class_='text-start')[3].text.strip() if len(row.find_all('td', class_='text-start')) > 3 else ''
                
                # Extract industry
                industry_cell = row.find_all('td', class_='text-start')[4]
                industry = industry_cell.find('a').text.strip() if industry_cell.find('a') else ''
                
                # Extract employee count
                employee_count = row.find_all('td', class_='text-start')[5].text.strip() if len(row.find_all('td', class_='text-start')) > 5 else ''
                
                # Extract revenue
                revenue = row.find_all('td', class_='text-start')[6].text.strip() if len(row.find_all('td', class_='text-start')) > 6 else ''
                
                # Extract growth rate
                growth_rate = row.find_all('td', class_='text-start')[7].text.strip() if len(row.find_all('td', class_='text-start')) > 7 else ''
                
                # Extract key person name and link
                key_person_cell = row.find_all('td', class_='text-start')[8]
                key_person = key_person_cell.find('a').text.strip() if key_person_cell.find('a') else ''
                key_person_link = key_person_cell.find('a')['href'] if key_person_cell.find('a') else ''
                
                # Extract position
                position = row.find_all('td', class_='text-start')[9].text.strip() if len(row.find_all('td', class_='text-start')) > 9 else ''
                
                # single startup page
                url_startup = base_url2[:-1] + company_link
                # Usa la funzione per inizializzare o resettare il driver
                driver = reset_driver()
                driver.get(url_startup)

                # prendo tutto il testo nella pagina della startup
                main_text = get_all_text(url_startup, only_main = True)

                # Wait for the exhibitors list to be present
                pageContent = driver.page_source

                # Parse the main page with BeautifulSoup
                soup = BeautifulSoup(pageContent, 'html.parser')

                # Extract body
                startup_body = soup.find('div', class_='tempCompany card-body')

                # Logo extraction
                rectangle = startup_body.find('div', id = 'revenue-financials')
                logo_element = rectangle.find('img', src=lambda x: x and "logo.clearbit.com" in x)
                logo_url = 'https:' + logo_element['src'] if logo_element else None
                
                # Extract company link
                company_link_element = rectangle.find('a', href=True)
                company_link = 'https:' + company_link_element['href'] if company_link_element else None
                
                # Employee extraction
                employee_internal_links = [base_url2[:-1] + a['href'] for a in soup.find_all('a', href=lambda x: x and "/employee/" in x)]
                employee_internal_links = ', '.join(employee_internal_links)

                # Append data to the list
                data.append([
                            company_name, country, location, funding,
                            industry, employee_count, revenue, growth_rate, key_person,
                            key_person_link, position, logo_url, company_link
                        ])

                new_data = pd.DataFrame(data, columns=["Startup Name", "Country", "City", "Total funding", "Industry", 
                                       "Number of employees", "Revenue", "Growth Rate", "Key Person", 
                                       "Key Person Link", "Position", "Logo URL", "Website URL"])

                # GPT-4 WORK
                initial_data = new_data.to_dict()
                url = new_data['Website URL'][0]

                text2 = get_all_text(url)
                text2 += main_text
                text_news = news([company_name])
                text2 += text_news
                print('get_all_text è andato OK')

                analysis_result = analyze_text_with_gpt(text2, initial_data)
                print('analyze_text_with_gpt è andato OK')

                # Convert the analysis result into a dictionary
                data_dict = convert_to_dict(analysis_result)
                print('convert_to_dict è andato OK')

                data_dict['google_news_urls'] = text_news

                data_dict['Key'] = generate_unique_key(data_dict['Website'])

                data_dict['timestamp'] = datetime.now().isoformat()
                
                # Convert the dictionary into a DataFrame
                df = pd.DataFrame([data_dict])

                # Upload the first record in 'df' to Airtable
                upload_record_to_airtable(df.iloc[0].to_dict(), url_airtable, headers)
                print('upload_record_to_airtable è andato OK')
                break
                
            except Exception as e:
                print(f"Failed to scrape row: {e}")

    except Exception as e:
        print(f"Failed to scrape page {page_num}: {e}")

### VENTURE RADAR

In [ ]:
# Initialize the WebDriver
driver = reset_driver()

# URL di base
base_url = "https://blog.ventureradar.com/"

# Apri la pagina con Selenium
driver.get(base_url)

# Simula lo scroll fino in fondo alla pagina per caricare nuovi articoli
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scorri verso il basso
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Aspetta il caricamento della pagina
    time.sleep(SCROLL_PAUSE_TIME)

    # Calcola l'altezza nuova della pagina dopo lo scroll
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # Esci dal ciclo se non ci sono nuovi contenuti da caricare
    if new_height == last_height:
        break
    last_height = new_height

# Ottieni il contenuto completo della pagina dopo aver caricato tutti gli articoli
pageContent = driver.page_source

# Usa BeautifulSoup per analizzare il contenuto della pagina
soup = BeautifulSoup(pageContent, 'html.parser')

# Trova tutti gli oggetti di tipo <article>
articles = soup.find_all('article')

# Lista per memorizzare i link trovati
links = []

# Itera su ogni <article> per estrarre il link
for article in articles:
    link = article.find('a', href=True)
    if link:
        links.append(link['href'])

filtered_links = analyze_links_with_gpt(links)


# Dizionario per memorizzare le informazioni di tutte le startup
startups_info = {}

# Itera su ogni link filtrato
for idx, link in enumerate(filtered_links):
    print(f"Processing link number: {idx+1} of {len(filtered_links)} total links")
    
    # Apri la pagina del link con Selenium
    # Usa la funzione per inizializzare o resettare il driver
    driver = reset_driver()
    driver.get(link)
    time.sleep(3)  # Attendi il caricamento della pagina
    
    # Ottieni il contenuto della pagina
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Estrai il contenuto del div con la classe "entry-content"
    entry_content_div = soup.find('div', class_='entry-content')
    
    if entry_content_div:
        # Converti il contenuto del div in una stringa HTML
        html_content = str(entry_content_div)
        
        # Analizza il contenuto HTML della pagina per estrarre le informazioni delle startup
        startup_data = analyze_startup_page_with_gpt(html_content, dict_example)
        
        # Aggiungi le informazioni al dizionario principale
        startups_info[f"page_{idx+1}"] = startup_data
    else:
        print(f"No entry-content found for link: {link}")

    break 

# Elaborazione dei dati delle startup
processed_startups_info = process_startups(startups_info)


# Itero sul dizionario così composto:
# {
# nome startup : {url website, url sito interno vendure},
# ...
# }

for startup_name, urls in processed_startups_info.items():
        print(f"Processing startup: {startup_name}")
        
        # Ottieni il testo completo dal sito web della startup
        startup_url = urls.get('Startup URL')
        if startup_url:
            startup_text = get_all_text(startup_url)
        else:
            startup_text = ""
        
        internal_startup_url = urls.get('Venture Radar URL')

        if internal_startup_url:
            #internal_startup_text = get_all_text(internal_startup_url, only_main=True)
            # Usa la funzione per inizializzare o resettare il driver
            driver = reset_driver()
            driver.get(internal_startup_url)
            time.sleep(3)  # Attendi il caricamento della pagina
    
            # Ottieni il contenuto della pagina
            page_content = driver.page_source
            soup = BeautifulSoup(page_content, 'html.parser')
    
            # Estrai il contenuto del div con la classe "entry-content"
            internal_startup_text = soup.find('div', class_='container c_d_ProfileContainer')

        else:
            internal_startup_text = ""

        # Ottieni le notizie relative alla startup
        news_summary = news([startup_name])  

        # Concatena le informazioni per l'analisi GPT
        full_text = f"Startup Information:\n{startup_text}\n\nNews Summary:\n{news_summary}"
        
        # Analizza il testo concatenato usando GPT
        gpt_analysis = analyze_text_with_gpt(full_text, initial_data = {internal_startup_text})
        data_dict = convert_to_dict(gpt_analysis)
        
        data_dict['google_news_urls'] = news_summary
        data_dict['Key'] = generate_unique_key(startup_url)

        # Convert the dictionary into a DataFrame
        df = pd.DataFrame([data_dict])

        # Upload the first record in 'df' to Airtable
        upload_record_to_airtable(df.iloc[0].to_dict(), url_airtable, headers)
        print('upload_record_to_airtable è andato OK')
        
        break            


### STARTUP RANKING

In [ ]:
# Initialize the WebDriver
driver = reset_driver()

# URL to scrape
url = "https://www.startupranking.com/startup/new"

# Open the webpage
driver.get(url)

# Allow some time for the page to load completely
time.sleep(3)

# Get the page source
page_content = driver.page_source

# Parse the page content with BeautifulSoup
soup = BeautifulSoup(page_content, 'html.parser')

# Find the table element by class name
table = soup.find('table', class_='table-striped wide new-startups')

# Check if the table was found
if table:
    # Prepare a list to hold the data
    data = []

    # Extract rows from the table
    rows = table.find_all('tr')
    for idx, row in enumerate(rows[1:]):
        # Extract the <div> with class "name" from each row
        name_div = row.find('div', class_='name')
        if name_div:
            # Get the text content for the startup name
            name = name_div.get_text(strip=True)
            
            # Get the link to the startup's profile
            link = name_div.find('a', href=True)['href']
            full_link = f"https://www.startupranking.com{link}"
            
            # Visit the startup's profile page
            driver.get(full_link)
            time.sleep(2)  # Allow time for the page to load

            # Get the page content
            profile_page_content = driver.page_source
            profile_soup = BeautifulSoup(profile_page_content, 'html.parser')
            
            # Extract the content from the div with class 'wrapper container group'
            wrapper_div = profile_soup.find('div', class_='wrapper container group')
            if wrapper_div:
                html_content = str(wrapper_div)

            # estraggo il sito web della startup e tutto il testo
            website = wrapper_div.find('p', class_ = "su-loc").find('a', href=True).text
            if website:
                startup_text = get_all_text(website)
            else:
                startup_text = ""

            # Ottieni le notizie relative alla startup
            news_summary = news([name]) 
            
            # Concatena le informazioni per l'analisi GPT
            full_text = f"Startup Information:\n{startup_text}\n\nNews Summary:\n{news_summary}\n\nHTML of Startup profile:\n{html_content}"

            # Analyze the content with GPT
            analysis_result = analyze_text_with_gpt(full_text, [])
            data_dict = convert_to_dict(analysis_result)
        
            data_dict['google_news_urls'] = news_summary
            data_dict['Key'] = generate_unique_key(website)

            # Convert the dictionary into a DataFrame
            df = pd.DataFrame([data_dict])

            # Upload the first record in 'df' to Airtable
            upload_record_to_airtable(df.iloc[0].to_dict(), url_airtable, headers)
            print('upload_record_to_airtable è andato OK')
                    
            break

else:
    print("Table not found!")

# Quit the WebDriver
driver.quit()

### STARTUPEABLE

In [ ]:
# Initialize the WebDriver
driver = reset_driver()

# Prepare a list to hold the data
data = []

# Iterate over the pages
for page in range(1, 97):  # Pages range from 1 to 96
    # URL to scrape
    url = f"https://startupeable.com/directorio/explora/?pg={page}&sort=a-z"

    # Open the webpage
    driver.get(url)

    # Allow some time for the page to load completely
    time.sleep(3)

    # Get the page source
    page_content = driver.page_source

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(page_content, 'html.parser')

    # Find the main div containing the "table"
    results_view = soup.find('div', class_='row results-view grid fc-type-2-results')

    # Check if the div was found
    if results_view:
        # Extract each startup block within the results-view div
        startups = results_view.find_all('div', class_='grid-item')

        for startup in startups:
            # Extract the name from the h4 tag
            name_tag = startup.find('h4', class_='case27-secondary-text listing-preview-title')
            name = name_tag.get_text(strip=True) if name_tag else None

            # Extract the link to the startup's profile
            link_tag = startup.find('a', href=True)
            profile_link = link_tag['href'] if link_tag else None

            # Add the name and link to the data list
            if name and profile_link:
                data.append({'Name': name, 'Profile Link': profile_link})
            

            # Visit the startup's profile page
            driver = reset_driver()
            driver.get(profile_link)
            time.sleep(2)  # Allow time for the page to load

            # Get the page content
            profile_page_content = driver.page_source
            profile_soup = BeautifulSoup(profile_page_content, 'html.parser')
            
            # Extract the content from the div with class 'wrapper container group'
            wrapper_div = profile_soup.find('div', class_='single-job-listing')
            if wrapper_div:
                html_content = str(wrapper_div)
            else:
                html_content = ""

            # estraggo il sito web della startup e tutto il testo
            website = wrapper_div.find('a', rel="nofollow", target="_blank")['href']
            if website:                
                startup_text = get_all_text(website)
            else:
                startup_text = ""

                       # Ottieni le notizie relative alla startup
            news_summary = news([name]) 
            
            # Concatena le informazioni per l'analisi GPT
            full_text = f"Startup Information:\n{startup_text}\n\nNews Summary:\n{news_summary}\n\nHTML of Startup profile:\n{html_content}"

            # Analyze the content with GPT
            analysis_result = analyze_text_with_gpt(full_text, [])
            data_dict = convert_to_dict(analysis_result)
        
            data_dict['google_news_urls'] = news_summary
            data_dict['Key'] = generate_unique_key(website)

            # Convert the dictionary into a DataFrame
            df = pd.DataFrame([data_dict])

            # Upload the first record in 'df' to Airtable
            upload_record_to_airtable(df.iloc[0].to_dict(), url_airtable, headers)
            print('upload_record_to_airtable è andato OK')
        


            break 
     
    else:
        print(f"No results-view found on page {page}!")

    break

# Quit the WebDriver
driver.quit()


### THE HUB

In [ ]:
# Initialize the WebDriver
driver = reset_driver()

# Prepare a list to hold the data
data = []

# Iterate over the pages
for page in range(1, 20):  # Pages range from 1 to 96
    # URL to scrape
    url = f"https://thehub.io/startups?industries=retail&page={page}"

    # Open the webpage
    driver.get(url)

    # Allow some time for the page to load completely
    time.sleep(3)

    # Get the page source
    page_content = driver.page_source

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(page_content, 'html.parser')

    # Find the main div containing the "table"
    results_view = soup.find_all('div', class_='mb-30 col-md-6 col-lg-4')

    # loop sui risultati trovati nella pagina
    for el in results_view:
        internal_link = "https://thehub.io" + el.find('a')['href']
        name = el.find('h4', class_ = 'card-title').text

        # Visit the startup's profile page
        driver = reset_driver()
        driver.get(internal_link)
        time.sleep(2)  # Allow time for the page to load

        # Get the page content
        profile_page_content = driver.page_source
        profile_soup = BeautifulSoup(profile_page_content, 'html.parser')
        
        # prendo html della pagina interna
        wrapper_div = profile_soup.find('div', class_ = 'company-form-preview')
        if wrapper_div:
            html_content = str(wrapper_div)
        else:
            html_content = ""

        # estraggo il sito web della startup e tutto il testo
        website = wrapper_div.find('a', class_ = 'text-blue-900')['href']
        if website:                
            startup_text = get_all_text(website)
        else:
            startup_text = ""

        # Ottieni le notizie relative alla startup
        news_summary = news([name]) 
            
        # Concatena le informazioni per l'analisi GPT
        full_text = f"Startup Information:\n{startup_text}\n\nNews Summary:\n{news_summary}\n\nHTML of Startup profile:\n{html_content}"

        # Analyze the content with GPT
        analysis_result = analyze_text_with_gpt(full_text, [])
        data_dict = convert_to_dict(analysis_result)
        
        data_dict['google_news_urls'] = news_summary
        data_dict['Key'] = generate_unique_key(website)

        # Convert the dictionary into a DataFrame
        df = pd.DataFrame([data_dict])

        # Upload the first record in 'df' to Airtable
        upload_record_to_airtable(df.iloc[0].to_dict(), url_airtable, headers)
        print('upload_record_to_airtable è andato OK')    

        
        break

        
    break 

### WELLFOUND

In [ ]:
# Initialize the WebDriver
for page_number in range(1, 2801):
    driver = initialize_driver(headless=True)
    # Vai alla pagina web
    driver.get(f"https://wellfound.com/startups?page={page_number}")

    # Simula comportamenti umani
    time.sleep(random.uniform(5, 10))  # Pausa iniziale casuale
    human_like_scroll(driver, pause_time=random.uniform(2, 5))  # Scorrimento umano
    print('human_like_scroll OK')
    safe_mouse_movement(driver)  # Movimento del mouse su un elemento sicuro
    print('safe_mouse_movement OK')
    random_click(driver)  # Click su un elemento interattivo
    print('random_click OK')

    # Attendi che la pagina si carichi completamente
    time.sleep(random.uniform(5, 10))

    # Estrai il contenuto della pagina
    page_content = driver.page_source

    # Chiudi il browser
    driver.quit()

    # Ora puoi usare BeautifulSoup su `page_content`
    soup = BeautifulSoup(page_content, 'html.parser')

    # Aggiungi una pausa casuale tra le iterazioni per ridurre il rischio di essere bloccato
    time.sleep(random.uniform(10, 20))

    container = soup.find("div", class_ = "mt-4 flex w-full flex-col gap-6")
    list_ = container.find_all("div", class_ = "rounded-lg border border-gray-400 p-8 pb-0")
    for el in list_ : 
        internal_link = "https://wellfound.com/" + el.find('a', class_ = "styles_component__UCLp3 styles_defaultLink__eZMqw !no-underline")["href"]
        name = el.find("header", class_ = "text-dark-aaaa font-medium antialiased text-lg").find_all('div')[0].text
        
        # Visit the startup's profile page overview
        driver = initialize_driver(headless=True)
        driver.get(internal_link)
        time.sleep(random.uniform(5, 15))  # Allow time for the page to load
        profile_page_content = driver.page_source
        driver.quit()


        profile_soup = BeautifulSoup(profile_page_content, 'html.parser')
        wrapper_div_overview = profile_soup.find('div', class_ = 'styles_box___OsDD')
        if wrapper_div_overview:
            html_content_overview = str(wrapper_div_overview)
        else:
            html_content_overview = "" 

        
        # Visit the startup's profile page funding
        internal_link_funding = internal_link + "/funding"

        driver = initialize_driver(headless=True)
        driver.get(internal_link_funding)
        time.sleep(random.uniform(5, 15))  # Allow time for the page to load
        profile_page_content = driver.page_source
        driver.quit()

        profile_soup = BeautifulSoup(profile_page_content, 'html.parser')
        wrapper_div_funding = profile_soup.find('div', class_ = "styles_component___DsQw")
        if wrapper_div_funding:
            html_content_funding = str(wrapper_div_funding)
        else:
            html_content_funding = "" 

        html_content = html_content_overview + html_content_funding

        # estraggo il sito web della startup e tutto il testo
        website = wrapper_div_overview.find('button', class_ = "styles_websiteLink___Rnfc").text
        if website:                
            startup_text = get_all_text(website)
        else:
            startup_text = ""

        # Ottieni le notizie relative alla startup
        news_summary = news([name]) 
                
        # Concatena le informazioni per l'analisi GPT
        full_text = f"Startup Information:\n{startup_text}\n\nNews Summary:\n{news_summary}\n\nHTML of Startup profile:\n{html_content}"

        # Analyze the content with GPT
        analysis_result = analyze_text_with_gpt(full_text, [])
        data_dict = convert_to_dict(analysis_result)
            
        data_dict['google_news_urls'] = news_summary
        data_dict['Key'] = generate_unique_key(website)

        # Convert the dictionary into a DataFrame
        df = pd.DataFrame([data_dict])

        # Upload the first record in 'df' to Airtable
        upload_record_to_airtable(df.iloc[0].to_dict(), url_airtable, headers)
        print('upload_record_to_airtable è andato OK')
        break 

### AI STARTUPS EUROPE

In [31]:
# Iterate over the pages
for page in range(1, 125):  # Pages range from 1 to 96
    # URL to scrape
    url = f"https://www.ai-startups-europe.eu//p{page}"

    # Open the webpage
    # Initialize the WebDriver
    driver = reset_driver()
    driver.get(url)

    # Allow some time for the page to load completely
    time.sleep(3)

    # Get the page source
    page_content = driver.page_source

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(page_content, 'html.parser')

    container = soup.find('ul', class_ = "startups-list").find_all('li')

    for el in container:
        logo = "https://www.ai-startups-europe.eu" + el.find('img')['src']
        name = el.find('h2').text
        website = el.find('a', class_ = "meta-item fw-bold stretched-link")["href"]

        initial_info = {"name": name, 
                        "logo": logo,
                        "website_url": website
                        }


        if website:                
            startup_text = get_all_text(website)
        else:
            startup_text = ""

        # Ottieni le notizie relative alla startup
        news_summary = news([name]) 
            
        # Concatena le informazioni per l'analisi GPT
        full_text = f"Startup Information:\n{startup_text}\n\nNews Summary:\n{news_summary}"

        # Analyze the content with GPT
        analysis_result = analyze_text_with_gpt(full_text, initial_info)
        data_dict = convert_to_dict(analysis_result)
        
        data_dict['google_news_urls'] = news_summary
        data_dict['Key'] = generate_unique_key(website)

        # Convert the dictionary into a DataFrame
        df = pd.DataFrame([data_dict])

        # Upload the first record in 'df' to Airtable
        upload_record_to_airtable(df.iloc[0].to_dict(), url_airtable, headers)
        print('upload_record_to_airtable è andato OK')


        break 

    break

Processing 10AI GmbH...


### CHECK INTEGRATED_TABLE 

In [ ]:
import requests
import json
import pandas as pd


# Airtable API Key e Base ID
api_key = 'patRHlt60PBL6jlV5.f2b9afed49bf23d9ab73cee6b283af76d3b4a9ebc0619002c6f104cc84382f2c'
base_id = 'appUMz37tbS84AKX4'
table_name = 'integrated_table'

# Endpoint dell'API di Airtable
url = f'https://api.airtable.com/v0/{base_id}/{table_name}'


# Intestazioni per la richiesta API
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

def get_all_records(url, headers):
    all_records = []
    params = {"pageSize": 100}
    
    while True:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            break
        
        records = response.json().get('records', [])
        all_records.extend(records)
        
        # Verifica se c'è un'altra pagina di record
        offset = response.json().get('offset')
        if not offset:
            break
        params['offset'] = offset

    return all_records

# Ottieni tutti i record
records = get_all_records(url, headers)

# Converti i record in un DataFrame pandas per una visualizzazione migliore
if records:
    df = pd.DataFrame([record['fields'] for record in records])
    print(df)
else:
    print("Nessun record trovato.")


In [21]:
import psycopg2
from psycopg2 import sql

# Define your connection parameters
DB_HOST = '35.231.104.140'
DB_PORT = '5432'  # Default PostgreSQL port
DB_NAME = 'rh_ai_storage'
DB_USER = 'kevin_capano'
DB_PASSWORD = '56LXhzMhTa9a'

# Function to establish a connection to the PostgreSQL database
def connect_db():
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        return conn
    except Exception as e:
        print(f"Unable to connect to the database: {e}")
        return None


def create_filter_query(table_name, filters):
    # Start building the query
    query = sql.SQL("SELECT * FROM {table} WHERE ").format(
        table=sql.Identifier(table_name)
    )

    filter_clauses = []
    filter_values = []

    for field, condition in filters.items():
        if isinstance(condition, str) and any(op in condition for op in ["<", ">", "<=", ">=", "!="]):
            operator, value = condition.split(" ", 1)
            filter_clauses.append(sql.SQL("{field} {operator} %s").format(
                field=sql.Identifier(field),
                operator=sql.SQL(operator)
            ))
            filter_values.append(value)
        else:
            filter_clauses.append(sql.SQL("{field} = %s").format(
                field=sql.Identifier(field)
            ))
            filter_values.append(condition)

    query = query + sql.SQL(" AND ").join(filter_clauses)

    return query, filter_values

def query_db(conn, table_name, filters):
    query, filter_values = create_filter_query(table_name, filters)
    
    try:
        with conn.cursor() as cursor:
            cursor.execute(query, filter_values)
            results = cursor.fetchall()
            return results
    except Exception as e:
        print(f"Error executing query: {e}")
        return None


def insert_record(conn, table_name, record):
    fields = sql.SQL(', ').join(map(sql.Identifier, record.keys()))
    values = sql.SQL(', ').join(sql.Placeholder() * len(record))

    query = sql.SQL("INSERT INTO {table} ({fields}) VALUES ({values}) RETURNING id").format(
        table=sql.Identifier(table_name),
        fields=fields,
        values=values
    )

    try:
        with conn.cursor() as cursor:
            cursor.execute(query, list(record.values()))
            conn.commit()
            inserted_id = cursor.fetchone()[0]
            return inserted_id
    except Exception as e:
        print(f"Error inserting record: {e}")
        conn.rollback()
        return None


In [ ]:
# Establish connection
conn = connect_db()

# Example: Query the database with multiple filters
filters = {
    "name":'Nome Azienda'
}

results = query_db(conn, "companies", filters)

if results:
    print("Query Results:")
    for row in results:
        print(row)

In [ ]:
# Establish connection
conn = connect_db()

# Example: Insert a new record
new_record = {
    "id": 1,
    "name": "OpenAI"
}

inserted_id = insert_record(conn, "companies", new_record)

if inserted_id:
    print(f"New record inserted with ID: {inserted_id}")

# Close connection
if conn:
    conn.close()
